In [1]:
import sys
sys.path.append('../..')
from utilities.data_manager import ExchangeDataManager
import pandas as pd
import ccxt
import ta
import seaborn as sns
import plotly.express as px
import pygwalker as pyg

In [2]:
ex = ccxt.binanceusdm()
markets = ex.load_markets()
pair_list = [m for m in markets if markets[m]["quote"] == 'USDT' and markets[m]["active"] is True]
len(pair_list)

309

In [ ]:
exchange = ExchangeDataManager(
    exchange_name="binanceusdm", path_download="../database/exchanges"
)

# To uncomment if you need to download data (it will take a while)
# await exchange.download_data(
#     coins=pair_list,
#     intervals=["1d"],
# )

In [4]:
def top_bot_strategy(period, top_bot_n, column_name, long_top=True, sides=["long", "short"]):
    INITIAL_WALLET = 1000
    FEES = 0.0006

    close_dict = {}
    volume_dict = {}
    change_dict = {}
    volume_evol_dict = {}
    volatility_dict = {}
    sharpe_dict = {}
    rsi_dict = {}
    rsi_evol_dict = {}
    ma50_dict = {}
    ma50_evol_dict = {}
    ma_distance_dict = {}

    for pair in pair_list:
        try:
            dft = exchange.load_data(pair, "1d")
            close_dict[f"{pair}_close"] = dft["close"]
            volume_dict[f"{pair}_volume"] = dft["volume"]
            change_dict[f"{pair}_change"] = dft["close"].pct_change(period)
            volume_evol_dict[f"{pair}_volume_evol"] = dft["volume"].pct_change(period)
            volatility_dict[f"{pair}_volatility"] = dft["close"].rolling(period).std() / dft["close"].rolling(period).mean()
            rsi_dict[f"{pair}_rsi"] = ta.momentum.RSIIndicator(dft["close"], period).rsi()
            rsi_evol_dict[f"{pair}_rsi_evol"] = ta.momentum.RSIIndicator(dft["close"], period).rsi().pct_change(period, fill_method=None)
            ma50_dict[f"{pair}_ma50"] = dft["close"].rolling(50).mean()
            ma50_evol_dict[f"{pair}_ma50_evol"] = dft["close"].rolling(50).mean().pct_change(period, fill_method=None)
            ma_distance_dict[f"{pair}_ma_distance"] = 100 * (dft["close"] - dft["close"].rolling(50).mean()) / dft["close"].rolling(50).mean()
            sharpe_dict[f"{pair}_sharpe"] = (
                period ** 0.5 
                * dft["close"].pct_change(period).rolling(period).mean() 
                / dft["close"].pct_change(period).rolling(period).std()
            )
        except Exception as e:
            # print(f"Error loading {pair}: {e}")
            pass

    # Convertir les dictionnaires en DataFrames
    close_df = pd.DataFrame(close_dict)
    volume_df = pd.DataFrame(volume_dict)
    change_df = pd.DataFrame(change_dict)
    volume_evol_df = pd.DataFrame(volume_evol_dict)
    volatility_df = pd.DataFrame(volatility_dict)
    rsi_df = pd.DataFrame(rsi_dict)
    rsi_evol_df = pd.DataFrame(rsi_evol_dict)
    ma50_evol_df = pd.DataFrame(ma50_evol_dict)
    ma_distance_df = pd.DataFrame(ma_distance_dict)
    sharpe_df = pd.DataFrame(sharpe_dict)

    # Concaténer tous les DataFrames en un seul
    df = pd.concat(
        [close_df, volume_df, change_df, volume_evol_df, volatility_df, rsi_df, sharpe_df, rsi_evol_df, ma50_evol_df, ma_distance_df], 
        axis=1
    )

    volume_columns = [c for c in df.columns if c.endswith("USDT_volume")]
    volume_evol_columns = [c for c in df.columns if c.endswith("USDT_volume_evol")]
    change_columns = [c for c in df.columns if c.endswith("USDT_change")]
    volatility_columns = [c for c in df.columns if c.endswith("USDT_volatility")]
    sharpe_columns = [c for c in df.columns if c.endswith("USDT_sharpe")]
    rsi_columns = [c for c in df.columns if c.endswith("USDT_rsi")]
    rsi_evol_columns = [c for c in df.columns if c.endswith("USDT_rsi_evol")]
    ma50_evol_columns = [c for c in df.columns if c.endswith("USDT_ma50_evol")]
    ma_distance_columns = [c for c in df.columns if c.endswith("USDT_ma_distance")]

    count = 0
    positions = {}
    trades = []
    balance = INITIAL_WALLET
    for index, row in df.iterrows():
        count += 1
        if count % period != 0:
            continue
        non_nan_close = row[change_columns].notna().sum()
        if non_nan_close < 50:
            continue
        
        if len(positions.keys()) > 0:
            for pair in positions.copy():
                position = positions[pair]
                close_price = row[f"{pair}_close"]
                side = position["side"]
                if side == "long":
                    trade_pct = (close_price - position["open_price"]) / position["open_price"]
                else:
                    trade_pct = (position["open_price"] - close_price) / position["open_price"]
                
                trade_pnl_without_fees = position["open_size"] * trade_pct
                close_size = position["open_size"] + trade_pnl_without_fees
                close_fees = close_size * FEES
                trade_pnl = trade_pnl_without_fees - close_fees - position["open_fees"]          
                balance += trade_pnl_without_fees - close_fees 

                trades.append(
                    {
                        "pair": pair,
                        "side": side,
                        "open_price": position["open_price"],
                        "close_price": close_price,
                        "open_size": position["open_size"],
                        "close_size": close_size,
                        "open_fees": position["open_fees"],
                        "close_fees": close_fees,
                        "open_time": position["open_time"],
                        "close_time": index,
                        "trade_pct": trade_pct,
                        "trade_pnl": trade_pnl,
                        "open_stat": position["open_stat"],
                    }
                )
                del positions[pair]

        current_changes = row[locals()[column_name]]

        if long_top:
            best_changes = current_changes.nlargest(top_bot_n)
            worst_changes = current_changes.nsmallest(top_bot_n)
        else:
            best_changes = current_changes.nsmallest(top_bot_n)
            worst_changes = current_changes.nlargest(top_bot_n)

        change_types = []
        if "long" in sides:
            change_types.append((best_changes, "long"))
        if "short" in sides:
            change_types.append((worst_changes, "short"))

        for changes, pos_type in change_types:
            for pair_column in changes.index:
                pair = pair_column.split("_")[0]
                close_price = row[f"{pair}_close"]
                usd_size = balance / (len(best_changes) + len(worst_changes))
                fees = usd_size * FEES
                balance -= fees
                positions[pair] = {
                    "side": pos_type,
                    "open_size": usd_size,
                    "open_price": close_price,
                    "open_fees": fees,
                    "open_time": index,
                    "open_stat": changes.loc[pair_column],
                }
        
    df_trades = pd.DataFrame(trades)
    print(f"Final balance: {balance}")
    return df_trades

In [46]:
# df = top_bot_strategy(
#     period=7, 
#     top_bot_n=5, 
#     column_name="change_columns", 
#     long_top=True, 
#     sides=["long"]
# )

# df = top_bot_strategy(
#     period=14, 
#     top_bot_n=2, 
#     column_name="rsi_evol_columns", 
#     long_top=True, 
#     sides=["long", "short"]
# )

# df = top_bot_strategy(
#     period=7, 
#     top_bot_n=2, 
#     column_name="volume_evol_columns", 
#     long_top=True, 
#     sides=["long", "short"]
# )

# df = top_bot_strategy(
#     period=30, 
#     top_bot_n=10, 
#     column_name="ma50_evol_columns", 
#     long_top=False, 
#     sides=["long", "short"]
# )

# df = top_bot_strategy(
#     period=7, 
#     top_bot_n=2, 
#     column_name="rsi_columns", 
#     long_top=True, 
#     sides=["long", "short"]
# )

# df = top_bot_strategy(
#     period=2, 
#     top_bot_n=10, 
#     column_name="ma_distance_columns", 
#     long_top=True, 
#     sides=["long"]
# )

df = top_bot_strategy(
    period=30, 
    top_bot_n=5, 
    column_name="volatility_columns", 
    long_top=False, 
    sides=["long"]
)


df.iloc[:20]

Final balance: 6101.345929304818


,pair,side,open_price,close_price,open_size,close_size,open_fees,close_fees,open_time,close_time,trade_pct,trade_pnl,open_stat
0,DOGE/USDT:USDT,long,0.002568,0.003568,100.000000,138.940810,0.060000,0.083364,2020-10-31,2020-11-30,0.389408,38.797445,0.012640
1,TRX/USDT:USDT,long,0.025790,0.032410,99.994000,125.661324,0.059996,0.075397,2020-10-31,2020-11-30,0.256689,25.531931,0.020690
2,EOS/USDT:USDT,long,2.528000,3.263000,99.988000,129.058879,0.059993,0.077435,2020-10-31,2020-11-30,0.290744,28.933450,0.025852
3,XRP/USDT:USDT,long,0.239600,0.664800,99.982001,277.412497,0.059989,0.166447,2020-10-31,2020-11-30,1.774624,177.204059,0.027579
4,ETC/USDT:USDT,long,5.274000,6.749000,99.976002,127.936678,0.059986,0.076762,2020-10-31,2020-11-30,0.279674,27.823928,0.027643
5,FIL/USDT:USDT,long,30.440000,21.445000,129.829081,91.464673,0.077897,0.054879,2020-11-30,2020-12-30,-0.295499,-38.497184,0.033393
6,MKR/USDT:USDT,long,569.330000,576.510000,129.821292,131.458509,0.077893,0.078875,2020-11-30,2020-12-30,0.012611,1.480449,0.044604
7,XMR/USDT:USDT,long,130.560000,159.630000,129.813502,158.717290,0.077888,0.095230,2020-11-30,2020-12-30,0.222656,28.730669,0.049531
8,THETA/USDT:USDT,long,0.642200,1.588800,129.805714,321.138769,0.077883,0.192683,2020-11-30,2020-12-30,1.473996,191.062489,0.049891
9,BNB/USDT:USDT,long,31.535000,38.224000,129.797925,157.329821,0.077879,0.094398,2020-11-30,2020-12-30,0.212114,27.359619,0.053477


In [47]:
df_sorted = df.sort_values('close_time')
df_sorted['cumulative_pnl'] = df_sorted['trade_pnl'].cumsum()

fig = px.line(df_sorted, x='close_time', y='cumulative_pnl', title='PnL Cumulatif au Fil du Temps')
fig.show()

In [37]:
df['date'] = df['close_time'].dt.date
daily_pnl = df.groupby(['date', 'side'])['trade_pnl'].sum().reset_index()

fig = px.bar(daily_pnl, x='date', y='trade_pnl', color='side',
             title='PnL Quotidien par Type de Trade',
             labels={'trade_pnl': 'PnL', 'date': 'Date'})
fig.show()


In [26]:
# 1. Agréger le PnL par paire et compter le nombre de trades
agg_df = df.groupby('pair').agg(
    total_pnl=('trade_pnl', 'sum'),
    trade_count=('trade_pnl', 'count')
).reset_index()

# 2. Trier les paires par PnL
agg_sorted = agg_df.sort_values(by='total_pnl', ascending=False)

# 3. Sélectionner les 5 paires avec le plus de PnL et les 5 avec le moins de PnL
# Si vous avez moins de 10 paires, ajustez le nombre en conséquence
top5 = agg_sorted.head(5)
bottom5 = agg_sorted.tail(5)

# 4. Combiner les paires sélectionnées
selected_pairs = pd.concat([top5, bottom5])

# 5. Trier pour une meilleure visualisation (optionnel)
selected_pairs = selected_pairs.sort_values(by='total_pnl', ascending=True)

# 6. Créer le graphique en barres avec Plotly
fig = px.bar(
    selected_pairs,
    x='pair',
    y='total_pnl',
    color='total_pnl',
    color_continuous_scale=[
        'red' if pnl < 0 else 'green' for pnl in selected_pairs['total_pnl']
    ],
    title='Top 5 et Bottom 5 Paires par PnL',
    labels={
        'pair': 'Paire de Trading',
        'total_pnl': 'PnL Total'
    },
    text='total_pnl',  # Affiche les valeurs de PnL sur les barres
    hover_data={'trade_count': True}  # Inclure le nombre de trades dans le hover
)

# Personnaliser le graphique
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')

# Ajuster la hauteur du graphique
fig.update_layout(
    xaxis_title="Paire de Trading",
    yaxis_title="PnL Total",
    coloraxis_showscale=False,  # Masquer l'échelle des couleurs
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    height=600  # Augmentez la hauteur selon vos besoins
)

# Afficher le graphique
fig.show()

In [27]:
walker = pyg.walk(df)

Box(children=(HTML(value='\n<div id="ifr-pyg-00062640fea3b90fxC938j4eDBO2orW6" style="height: auto">\n    <hea…